In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.gaussian_process.kernels import *
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

In [116]:
dftrain_x = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\X_train.csv')
dftrain_y = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\y_train.csv')
dftest_x = pd.read_csv(r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\X_test.csv')

In [117]:
#drop id 
dftrain_x.pop('id')
dftrain_y.pop('id')
dftest_x.pop('id')

0        0.0
1        1.0
2        2.0
3        3.0
4        4.0
       ...  
771    771.0
772    772.0
773    773.0
774    774.0
775    775.0
Name: id, Length: 776, dtype: float64

In [118]:
#imputation train
#model
imp = IterativeImputer(max_iter=40, random_state=0, n_nearest_features=100)
imp.fit(dftrain_x)
A = imp.transform(dftrain_x)
column_names = dftrain_x.columns
new_datax = pd.DataFrame(A)
new_datax.columns = column_names 

In [119]:
#imputation test
#model
imp = IterativeImputer(max_iter=40, random_state=0, n_nearest_features=100)
imp.fit(dftest_x)
A = imp.transform(dftest_x)
column_names = dftest_x.columns
dftest_x = pd.DataFrame(A)
dftest_x.columns = column_names 

In [332]:
#feature selection train
selector = SelectKBest(f_regression, k=30) #75 for gaussian processes
new_datax_reduced = selector.fit_transform(new_datax, np.array(dftrain_y).ravel())
cols = selector.get_support(indices=True)

new_dftest_x = np.array(dftest_x.iloc[:,cols])

C:\Users\sedri\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [333]:
#scale 
scaler = preprocessing.StandardScaler().fit(new_datax_reduced)
X_scaled = scaler.transform(new_datax_reduced)

scaler = preprocessing.StandardScaler().fit(new_dftest_x)
X_test_scaled = scaler.transform(new_dftest_x)

In [334]:
#split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, dftrain_y, test_size=0.2, random_state=42)

# Gaussian process

In [8]:
#train model - Gaussian process
kernel = Matern(nu=0.5) + DotProduct() + WhiteKernel()
GP_model = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=5, normalize_y=True).fit(X_train, y_train)

y_pred = []
y_std = []

for i in range(len(X_test)):
    mu, std = GP_model.predict(X_test[i].reshape(1, -1), return_std=True)
    y_pred.append(mu)
    y_std.append(std)
    
score = r2_score(y_test, np.array(y_pred).squeeze())
print("Gaussian Processes results: {}".format(score))

Gaussian Processes results: 0.527837398167956


In [9]:
#test - Gaussoian process
y_pred_test = []
y_std_test = []
for i in range(len(new_dftest_x)):
    mu, std = GP_model.predict(X_test_scaled[i].reshape(1, -1), return_std=True)
    y_pred_test.append(mu)
    y_std_test.append(std)

In [10]:
#save solution
solution = []
indeces = []
u = 0
for i in y_pred_test:
    solution.append(i[0][0])
    indeces.append(float(u))
    u+=1
    
df_solution = pd.DataFrame()
df_solution['id'] = indeces
df_solution['y'] = solution

In [11]:
df_solution

,id,y
0,0.0,63.736589
1,1.0,72.571069
2,2.0,67.315190
3,3.0,77.010111
4,4.0,77.339483
...,...,...
771,771.0,67.289574
772,772.0,72.070758
773,773.0,75.507419
774,774.0,67.911701


In [12]:
df_solution.to_csv(index=False, path_or_buf=r'C:\Users\sedri\Desktop\ETHZ\Master\3_HS22\Advanced Machine Learning\Projects\1_project\solution.csv')

# Neural Network

In [22]:
import tensorflow as tf

In [335]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(30,1)),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

In [336]:
model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_36 (Flatten)         (None, 30)                0         
_________________________________________________________________
dense_98 (Dense)             (None, 16)                496       
_________________________________________________________________
dropout_27 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 1)                 17        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [337]:
#random
predictions = model(X_train[:1]).numpy()

In [338]:
loss_fn = tf.keras.losses.MeanSquaredError()

In [339]:
#random
loss_fn(y_train[:1], predictions).numpy()

3959.074

In [340]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [341]:
model.fit(X_train, y_train, epochs=1500)

Epoch 1/1500
31/31 [==============================] - 0s 2ms/step - loss: 4843.4766 - accuracy: 0.0000e+00
Epoch 2/1500
31/31 [==============================] - 0s 2ms/step - loss: 4722.4941 - accuracy: 0.0000e+00
Epoch 3/1500
31/31 [==============================] - 0s 1ms/step - loss: 4569.4546 - accuracy: 0.0000e+00
Epoch 4/1500
31/31 [==============================] - 0s 1ms/step - loss: 4402.9268 - accuracy: 0.0000e+00
Epoch 5/1500
31/31 [==============================] - 0s 1ms/step - loss: 4192.2295 - accuracy: 0.0000e+00
Epoch 6/1500
31/31 [==============================] - 0s 2ms/step - loss: 3947.4138 - accuracy: 0.0000e+00
Epoch 7/1500
31/31 [==============================] - 0s 2ms/step - loss: 3718.5940 - accuracy: 0.0000e+00
Epoch 8/1500
31/31 [==============================] - 0s 2ms/step - loss: 3481.2834 - accuracy: 0.0000e+00
Epoch 9/1500
31/31 [==============================] - 0s 3ms/step - loss: 3234.4448 - accuracy: 0.0000e+00
Epoch 10/1500
31/31 [================

31/31 [==============================] - 0s 2ms/step - loss: 692.6936 - accuracy: 0.0000e+00
Epoch 77/1500
31/31 [==============================] - 0s 1ms/step - loss: 677.2348 - accuracy: 0.0000e+00
Epoch 78/1500
31/31 [==============================] - 0s 1ms/step - loss: 643.0619 - accuracy: 0.0000e+00
Epoch 79/1500
31/31 [==============================] - 0s 1ms/step - loss: 580.2660 - accuracy: 0.0000e+00
Epoch 80/1500
31/31 [==============================] - 0s 1ms/step - loss: 605.7676 - accuracy: 0.0000e+00
Epoch 81/1500
31/31 [==============================] - 0s 3ms/step - loss: 606.1076 - accuracy: 0.0000e+00
Epoch 82/1500
31/31 [==============================] - 0s 2ms/step - loss: 615.1075 - accuracy: 0.0000e+00
Epoch 83/1500
31/31 [==============================] - 0s 2ms/step - loss: 586.2375 - accuracy: 0.0000e+00
Epoch 84/1500
31/31 [==============================] - 0s 1ms/step - loss: 563.0716 - accuracy: 0.0000e+00
Epoch 85/1500
31/31 [==============================

31/31 [==============================] - 0s 2ms/step - loss: 188.6493 - accuracy: 0.0000e+00
Epoch 153/1500
31/31 [==============================] - 0s 1ms/step - loss: 188.6134 - accuracy: 0.0000e+00
Epoch 154/1500
31/31 [==============================] - 0s 1ms/step - loss: 191.0714 - accuracy: 0.0000e+00
Epoch 155/1500
31/31 [==============================] - 0s 1ms/step - loss: 198.1960 - accuracy: 0.0000e+00
Epoch 156/1500
31/31 [==============================] - 0s 2ms/step - loss: 197.7584 - accuracy: 0.0000e+00
Epoch 157/1500
31/31 [==============================] - 0s 1ms/step - loss: 188.9989 - accuracy: 0.0000e+00
Epoch 158/1500
31/31 [==============================] - 0s 2ms/step - loss: 172.2155 - accuracy: 0.0000e+00
Epoch 159/1500
31/31 [==============================] - 0s 1ms/step - loss: 188.9081 - accuracy: 0.0000e+00
Epoch 160/1500
31/31 [==============================] - 0s 1ms/step - loss: 192.4655 - accuracy: 0.0000e+00
Epoch 161/1500
31/31 [=====================

Epoch 228/1500
31/31 [==============================] - 0s 1ms/step - loss: 159.8897 - accuracy: 0.0000e+00
Epoch 229/1500
31/31 [==============================] - 0s 1ms/step - loss: 155.6980 - accuracy: 0.0000e+00
Epoch 230/1500
31/31 [==============================] - 0s 2ms/step - loss: 136.9560 - accuracy: 0.0000e+00
Epoch 231/1500
31/31 [==============================] - 0s 2ms/step - loss: 144.0371 - accuracy: 0.0000e+00
Epoch 232/1500
31/31 [==============================] - 0s 2ms/step - loss: 145.2116 - accuracy: 0.0000e+00
Epoch 233/1500
31/31 [==============================] - 0s 2ms/step - loss: 134.5226 - accuracy: 0.0000e+00
Epoch 234/1500
31/31 [==============================] - 0s 2ms/step - loss: 142.0889 - accuracy: 0.0000e+00
Epoch 235/1500
31/31 [==============================] - 0s 2ms/step - loss: 149.3381 - accuracy: 0.0000e+00
Epoch 236/1500
31/31 [==============================] - 0s 2ms/step - loss: 153.8139 - accuracy: 0.0000e+00
Epoch 237/1500
31/31 [======

31/31 [==============================] - 0s 3ms/step - loss: 143.0998 - accuracy: 0.0000e+00
Epoch 304/1500
31/31 [==============================] - 0s 2ms/step - loss: 141.2688 - accuracy: 0.0000e+00
Epoch 305/1500
31/31 [==============================] - 0s 2ms/step - loss: 136.5994 - accuracy: 0.0000e+00
Epoch 306/1500
31/31 [==============================] - 0s 1ms/step - loss: 126.9900 - accuracy: 0.0000e+00
Epoch 307/1500
31/31 [==============================] - 0s 1ms/step - loss: 136.4089 - accuracy: 0.0000e+00
Epoch 308/1500
31/31 [==============================] - 0s 1ms/step - loss: 142.7625 - accuracy: 0.0000e+00
Epoch 309/1500
31/31 [==============================] - 0s 1ms/step - loss: 146.5654 - accuracy: 0.0000e+00
Epoch 310/1500
31/31 [==============================] - 0s 1ms/step - loss: 143.9520 - accuracy: 0.0000e+00
Epoch 311/1500
31/31 [==============================] - 0s 1ms/step - loss: 137.0796 - accuracy: 0.0000e+00
Epoch 312/1500
31/31 [=====================

...

31/31 [==============================] - 0s 2ms/step - loss: 105.2259 - accuracy: 0.0000e+00
Epoch 1131/1500
31/31 [==============================] - 0s 2ms/step - loss: 109.4773 - accuracy: 0.0000e+00
Epoch 1132/1500
31/31 [==============================] - 0s 2ms/step - loss: 107.1892 - accuracy: 0.0000e+00
Epoch 1133/1500
31/31 [==============================] - 0s 2ms/step - loss: 104.0456 - accuracy: 0.0000e+00
Epoch 1134/1500
31/31 [==============================] - 0s 2ms/step - loss: 108.3227 - accuracy: 0.0000e+00
Epoch 1135/1500
31/31 [==============================] - 0s 2ms/step - loss: 107.9899 - accuracy: 0.0000e+00
Epoch 1136/1500
31/31 [==============================] - 0s 2ms/step - loss: 113.5833 - accuracy: 0.0000e+00
Epoch 1137/1500
31/31 [==============================] - 0s 2ms/step - loss: 108.9028 - accuracy: 0.0000e+00
Epoch 1138/1500
31/31 [==============================] - 0s 2ms/step - loss: 109.9217 - accuracy: 0.0000e+00
Epoch 1139/1500
31/31 [============

31/31 [==============================] - 0s 2ms/step - loss: 105.2838 - accuracy: 0.0000e+00
Epoch 1205/1500
31/31 [==============================] - 0s 2ms/step - loss: 108.0864 - accuracy: 0.0000e+00
Epoch 1206/1500
31/31 [==============================] - 0s 2ms/step - loss: 106.4029 - accuracy: 0.0000e+00
Epoch 1207/1500
31/31 [==============================] - 0s 1ms/step - loss: 94.5383 - accuracy: 0.0000e+00
Epoch 1208/1500
31/31 [==============================] - 0s 2ms/step - loss: 106.3785 - accuracy: 0.0000e+00
Epoch 1209/1500
31/31 [==============================] - 0s 2ms/step - loss: 100.9018 - accuracy: 0.0000e+00
Epoch 1210/1500
31/31 [==============================] - 0s 2ms/step - loss: 101.8592 - accuracy: 0.0000e+00
Epoch 1211/1500
31/31 [==============================] - 0s 1ms/step - loss: 110.7780 - accuracy: 0.0000e+00
Epoch 1212/1500
31/31 [==============================] - 0s 1ms/step - loss: 109.1331 - accuracy: 0.0000e+00
Epoch 1213/1500
31/31 [=============

31/31 [==============================] - 0s 2ms/step - loss: 105.9955 - accuracy: 0.0000e+00
Epoch 1280/1500
31/31 [==============================] - 0s 2ms/step - loss: 101.2052 - accuracy: 0.0000e+00
Epoch 1281/1500
31/31 [==============================] - 0s 2ms/step - loss: 99.6312 - accuracy: 0.0000e+00
Epoch 1282/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.4949 - accuracy: 0.0000e+00
Epoch 1283/1500
31/31 [==============================] - 0s 2ms/step - loss: 92.9015 - accuracy: 0.0000e+00
Epoch 1284/1500
31/31 [==============================] - 0s 2ms/step - loss: 102.2920 - accuracy: 0.0000e+00
Epoch 1285/1500
31/31 [==============================] - 0s 2ms/step - loss: 101.6689 - accuracy: 0.0000e+00
Epoch 1286/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.1459 - accuracy: 0.0000e+00
Epoch 1287/1500
31/31 [==============================] - 0s 2ms/step - loss: 101.4898 - accuracy: 0.0000e+00
Epoch 1288/1500
31/31 [================

31/31 [==============================] - 0s 2ms/step - loss: 101.3131 - accuracy: 0.0000e+00
Epoch 1355/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.3098 - accuracy: 0.0000e+00
Epoch 1356/1500
31/31 [==============================] - 0s 2ms/step - loss: 105.4210 - accuracy: 0.0000e+00
Epoch 1357/1500
31/31 [==============================] - 0s 2ms/step - loss: 103.0021 - accuracy: 0.0000e+00
Epoch 1358/1500
31/31 [==============================] - 0s 2ms/step - loss: 104.1736 - accuracy: 0.0000e+00
Epoch 1359/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.6993 - accuracy: 0.0000e+00
Epoch 1360/1500
31/31 [==============================] - 0s 2ms/step - loss: 98.3752 - accuracy: 0.0000e+00
Epoch 1361/1500
31/31 [==============================] - 0s 2ms/step - loss: 101.1421 - accuracy: 0.0000e+00
Epoch 1362/1500
31/31 [==============================] - 0s 2ms/step - loss: 105.8083 - accuracy: 0.0000e+00
Epoch 1363/1500
31/31 [===============

31/31 [==============================] - 0s 2ms/step - loss: 100.7889 - accuracy: 0.0000e+00
Epoch 1430/1500
31/31 [==============================] - 0s 2ms/step - loss: 100.7827 - accuracy: 0.0000e+00
Epoch 1431/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.4737 - accuracy: 0.0000e+00
Epoch 1432/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.9778 - accuracy: 0.0000e+00
Epoch 1433/1500
31/31 [==============================] - 0s 2ms/step - loss: 96.6607 - accuracy: 0.0000e+00
Epoch 1434/1500
31/31 [==============================] - 0s 2ms/step - loss: 94.5977 - accuracy: 0.0000e+00
Epoch 1435/1500
31/31 [==============================] - 0s 2ms/step - loss: 97.3849 - accuracy: 0.0000e+00
Epoch 1436/1500
31/31 [==============================] - 0s 2ms/step - loss: 98.5012 - accuracy: 0.0000e+00
Epoch 1437/1500
31/31 [==============================] - 0s 2ms/step - loss: 95.2246 - accuracy: 0.0000e+00
Epoch 1438/1500
31/31 [===================

In [342]:
y_pred = model(X_test[:]).numpy()

In [343]:
score = r2_score(y_test, np.array(y_pred).squeeze())
print("Neural Network results: {}".format(score))

Neural Network results: 0.37097146368676004


In [344]:
y_pred

array([[79.355576],
       [66.92178 ],
       [77.10847 ],
       [53.33678 ],
       [58.21122 ],
       [73.761734],
       [77.85413 ],
       [71.398994],
       [72.73436 ],
       [70.39867 ],
       [75.45062 ],
       [66.716064],
       [64.66222 ],
       [70.88791 ],
       [78.99096 ],
       [62.916103],
       [61.142166],
       [66.66062 ],
       [65.93121 ],
       [73.42964 ],
       [85.98444 ],
       [70.93959 ],
       [70.89068 ],
       [55.280003],
       [68.780304],
       [72.06676 ],
       [64.3414  ],
       [63.85195 ],
       [71.2101  ],
       [75.74546 ],
       [65.68766 ],
       [74.07683 ],
       [73.476036],
       [83.17357 ],
       [73.724365],
       [77.239105],
       [70.38646 ],
       [72.41591 ],
       [70.28409 ],
       [80.040184],
       [66.37242 ],
       [66.74439 ],
       [63.163013],
       [84.07362 ],
       [71.60813 ],
       [68.5872  ],
       [71.92581 ],
       [77.58944 ],
       [81.241615],
       [77.68375 ],
